<a href="https://colab.research.google.com/github/gursifath/Intro-to-Langchain/blob/main/Create_New_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://python.langchain.com/docs/tutorials/agents/

stack trace: https://smith.langchain.com/o/9940b792-ccc7-454c-8f50-5749603787ca/projects/p/58d64707-c8a4-4c0f-bbea-df4231831d87?timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0

In [ ]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

In [ ]:
!export LANGSMITH_TRACING="true"
!export LANGSMITH_API_KEY="..."

import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


In [ ]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········


# Define tools

In [ ]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751760432, 'localtime': '2025-07-05 17:07'}, 'current': {'last_updated_epoch': 1751760000, 'last_updated': '2025-07-05 17:00', 'temp_c': 15.2, 'temp_f': 59.4, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 13.6, 'wind_kph': 22.0, 'wind_degree': 264, 'wind_dir': 'W', 'pressure_mb': 1021.0, 'pressure_in': 30.14, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 88, 'cloud': 75, 'feelslike_c': 15.2, 'feelslike_f': 59.4, 'windchill_c': 12.9, 'windchill_f': 55.3, 'heatindex_c': 14.0, 'heatindex_f': 57.2, 'dewpoint_c': 12.1, 'dewpoint_f

In [ ]:
!pip install -qU "langchain[google-genai]"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
!pip uninstall -y google-ai-generativelanguage google-generativeai
!pip install -qU "langchain[google-genai]"

Found existing installation: google-ai-generativelanguage 0.6.18
Uninstalling google-ai-generativelanguage-0.6.18:
  Successfully uninstalled google-ai-generativelanguage-0.6.18
Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5


In [ ]:
# link to get API key: https://aistudio.google.com/app/u/5/apikey

import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [ ]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hi there! How can I help you today?'

In [ ]:
model_with_tools = model.bind_tools(tools)

In [ ]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hi, how can I help you today?

Tool calls: []


In [ ]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'Weather in San Francisco'}, 'id': '6e6fc4fc-5324-4727-9ce9-7872b2ac1a1b', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [ ]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I help you today?


In [ ]:
input_message = {"role": "user", "content": "Search for the weather in SF"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (8940aa7d-4c83-4496-959a-a24bf01a8065)
 Call ID: 8940aa7d-4c83-4496-959a-a24bf01a8065
  Args:
    query: Weather in San Francisco
================================= Tool Message =================================
Name: tavily_search

{"query": "Weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751760451, 'localtime': '2025-07-05 17:07'}, 'current': {'last_updated_epoch': 1751760000, 'last_updated': '2025-07-05 17:00', 'temp_c': 15.1, 'temp_f': 59.2, 'is_d

In [ ]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in SF
================================== Ai Message ==================================
Tool Calls:
  tavily_search (f1be9f27-5d05-4791-a837-c275c552ea07)
 Call ID: f1be9f27-5d05-4791-a837-c275c552ea07
  Args:
    query: Weather in San Francisco
================================= Tool Message =================================
Name: tavily_search

{"query": "Weather in San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751760451, 'localtime': '2025-07-05 17:07'}, 'current': {'last_updated_epoch': 1751760000, 'last_updated': '2025-07-05 17:00', 'temp_c': 15.1, 'temp_f': 59.2, 'is_d

In [ ]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The weather in San| Francisco is partly cloudy with a temperature of 63.0°F.| The wind is blowing from the west at 13.6 mph. The| humidity is 67%.|

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [39]:
input_message = {"role": "user", "content": "Hi, I'm GSB!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm GSB!
================================== Ai Message ==================================

Hi GSB! How can I assist you today?


In [40]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is GSB.
